In [2]:
from project import data_frames

for key, values in data_frames.items():
    print(key)

energy-data/power-laws-forecasting-energy-consumption-holidays.csv
energy-data/power-laws-forecasting-energy-consumption-metadata.csv
energy-data/power-laws-forecasting-energy-consumption-submission-forecast-period.csv
energy-data/power-laws-forecasting-energy-consumption-test-data.csv
energy-data/power-laws-forecasting-energy-consumption-training-data.csv
energy-data/power-laws-forecasting-energy-consumption-weather.csv
